Deep Autoencoder on MNIST

Exe. 1

In [ ]:
#Import the MNIST dataset from keras.datasets and load it in x train, y train, x test, y test variables.
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#check the train and test shape.
print(X_train.shape)
print(X_test.shape)
#In oder to be able using the sigmoid activation function, normalize x train and x test according to the maximum and minimum elements of image set, for instance check x train[0].
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

Exe. 2

In [ ]:
#Plot some images to see your normalization results.
import matplotlib.pyplot as plt
plt.imshow(X_train[0], cmap='gray')
plt.show()
plt.imshow(X_train[1], cmap='gray')
plt.show()
plt.imshow(X_train[2], cmap='gray')
plt.show()
# Notice The Encoder generally uses a series of Dense and/or Convolutional layers to encode an 
# image into a fixed length vector that represents the image acompact form, while the Decoder 
# uses Dense and/or Convolutional layers toconvert the latent representation vector back into 
# that same image or anothermodified image (see Figure 1).

# Latent size is the size of the latent space: the vector holding the information
# after compression. This value is a crucial hyperparameter. If this value is too
# small, there won’t be enough data for reconstruction and if the value is too
# large, overfitting can occur.

Exe. 3

In [1]:
# Let’s define the LATENT SIZE = 32. Create an encoder model consists of a series  
# of dense layers, each layer is followed by a Dropout and a ReLU layer.
Latent_Size = 32
from keras.layers import Dense, Flatten, LeakyReLU, ReLU, Activation, Dropout
from keras.models import Model, Sequential

encoder = Sequential(
    [
        Flatten(input_shape=(28, 28)),
        Dense(512),
        ReLU(),
        Dropout(0.5),
        Dense(256),
        ReLU(),
        Dropout(0.5),
        Dense(128),
        ReLU(),
        Dropout(0.5),
        Dense(64),
        ReLU(),
        Dropout(0.5),
        Dense(Latent_Size),
        ReLU(),
    ])

model.summary()

Exe. 4

In [ ]:
# Create a decoder model namely decoder. 
# The decoder is essentially the same as the encoder but in reverse.
# Dense, ReLU, Dropout, Dense, ReLU, Dropout, Dense, ReLU,
# Dropout, Dense, ReLU, Dropout, Dense, Acivation, Reshape
decoder = Sequential(
    [
        Dense(64, input_shape=(Latent_Size,)),
        ReLU(),
        Dropout(0.5),
        Dense(128),
        ReLU(),
        Dropout(0.5),
        Dense(256),
        ReLU(),
        Dropout(0.5),
        Dense(512),
        ReLU(),
        Dropout(0.5),
        Dense(784),
        Activation("sigmoid"),
        Reshape((28, 28)),
    ])
decoder.summary()

#decoder = Model(encoded, Flatten()(Dense(64)(LeakyReLU()(Dropout(0.3)(Dense(128)(LeakyReLU()(Dropout(0.3)(Dense(256)(LeakyReLU()(Dropout(0.3)(Dense(512)(LeakyReLU()(Dropout(0.3)(Dense(784)(LeakyReLU()(Dropout(0.3)(encoded)))))))))))))))))
#decoder.summary()

Exe. 5

In [ ]:
# implement the following code in your project:
from tensorflow.keras.layers import Input

img = Input(shape = (28 , 28))
latent_vector = encoder(img)
output = decoder(latent_vector)
model = Model(inputs = img, outputs = output)
model.compile("nadam", loss = "binary_crossentropy")

Exe. 6

In [ ]:
from IPython import display # If using IPython, Colab or Jupyter

EPOCHS = 20

for epoch in range(EPOCHS):
    fig, axs = plt.subplots(4, 4)
    rand = X_test[np.random.randint(0, 10000, 16)].reshape((4, 4, 1, 28, 28))
    
    display.clear_output() # If you imported display from IPython
    
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(model.predict(rand[i, j])[0], cmap = "gray")
            axs[i, j].axis("off")
    
    plt.subplots_adjust(wspace = 0, hspace = 0)
    plt.show()
    print("-----------", "EPOCH", epoch, "-----------")
    model.fit(X_train, X_train)

Denoising autoencoder on MNIST

Exe. 7

In [ ]:
# Generate corrupted MNIST images by adding noise with normal distribution 
# (mean = 0.5 and std= 0.5) to your x train and x test dataset. Fix
# the random seed with your student number.
import numpy as np
np.random.seed(2017011)
noise_factor = 0.5
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)


Exe. 8

In [ ]:
# After adding the random generated noises to the x sets, keep only those among 0 and 1 using np.clip()/.
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

Exe. 9

In [ ]:
#Print some of your noisy images to see how they are noisy now.
plt.imshow(X_train_noisy[0], cmap='gray')
plt.show()
plt.imshow(X_train_noisy[1], cmap='gray')
plt.show()

Exe. 10

In [ ]:
# Check the new noisy data with the previous model. How are the
# results? How they are close to the real images?
# check the noisy data with the previous model
EPOCHS = 5

for epoch in range(EPOCHS):
    fig, axs = plt.subplots(4, 4)
    rand = X_test_noisy[np.random.randint(0, 10000, 16)].reshape((4, 4, 1, 28, 28))
    
    display.clear_output() # If you imported display from IPython
    
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(model.predict(rand[i, j])[0], cmap = "gray")
            axs[i, j].axis("off")
    
    plt.subplots_adjust(wspace = 0, hspace = 0)
    plt.show()
    print("-----------", "EPOCH", epoch, "-----------")
    model.fit(X_train_noisy, X_train_noisy)